# 7章 ヒューリスティック

## 7.1 貪欲法

In [2]:
# A46
N = int(input())
X, Y = [None] * N, [None] * N
for i in range(N):
    X[i], Y[i] = map(int, input().split())
Current = 1
Passed = [False] * N
Passed[0] = True
print(1)
while (True):
    Next = 0
    kyori = 2*(10**3)
    for i in range(N):
        if (Passed[i] == False):
            i_kyori = (abs(X[i]-X[Current-1])**2 + abs(Y[i]-Y[Current-1])**2) ** 0.5
            if (kyori > i_kyori):
                kyori = i_kyori
                Next = i+1
    if (Next == 0):
        break
    Current = Next
    Passed[Current-1] = True
    print(Current)
print(1)

1
5
6
2
4
3
7
1


## 7.2 局所探索法

In [ ]:
# A46
import random
N = int(input())
X, Y = [None] * N, [None] * N
for i in range(N):
    X[i], Y[i] = map(int, input().split())
P = [i % N for i in range(N+1)]

def get_score(X, Y, P):
    score = 0
    for i in range(N):
        score += ((X[P[i]] - X[P[i+1]])**2 + (Y[P[i]] - Y[P[i+1]])**2) ** 0.5
    return score

LOOP = 500
current_score = get_score(X, Y, P)
for t in range(LOOP):
    l = random.randint(1, N-1)
    r = random.randint(1, N-1)
    if (l > r):
        l, r = r, l
    P[l:r+1] = reversed(P[l:r+1])
    new_score = get_score(X, Y, P)
    if (current_score >= new_score):
        current_score = new_score
    else:
        P[l:r+1] = reversed(P[l:r+1])

for i in P:
    print(i + 1)

1
2
6
7
3
4
5
1


## 7.3 焼きなまし法

In [2]:
# A46
import math
import random
N = int(input())
X, Y = [None] * N, [None] * N
for i in range(N):
    X[i], Y[i] = map(int, input().split())
P = [i % N for i in range(N+1)]

def get_score(X, Y, P):
    score = 0
    for i in range(N):
        score += ((X[P[i]] - X[P[i+1]])**2 + (Y[P[i]] - Y[P[i+1]])**2) ** 0.5
    return score

LOOP = 500
current_score = get_score(X, Y, P)
for t in range(LOOP):
    l = random.randint(1, N-1)
    r = random.randint(1, N-1)
    if (l > r):
        l, r = r, l
    P[l:r+1] = reversed(P[l:r+1])
    new_score = get_score(X, Y, P)
    T = 30 - 28 * (t / LOOP)
    probability = math.exp(min((current_score - new_score) / T, 0))
    if (random.random() < probability):
        current_score = new_score
    else:
        P[l:r+1] = reversed(P[l:r+1])

for i in P:
    print(i + 1)

1
7
2
6
5
4
3
1


## 7.4 ビームサーチ

In [3]:
# A49
import copy
T = int(input())
P, Q, R = [None] * T, [None] * T, [None] * T
for i in range(T):
    P[i], Q[i], R[i] = map(int, input().split())
Beam = [list() for _ in range(T+1)]
Beam[0] = [[0, [0] * 20, None, -1]]

for i in range(T):
    candidate = list()
    for j in range(len(Beam[i])):
        A = copy.deepcopy(Beam[i][j])
        A[1][P[i]-1] += 1
        A[1][Q[i]-1] += 1
        A[1][R[i]-1] += 1
        A[2] = 'A'
        A[3] = j
        A[0] += A[1].count(0)
        B = copy.deepcopy(Beam[i][j])
        B[1][P[i]-1] -= 1
        B[1][Q[i]-1] -= 1
        B[1][R[i]-1] -= 1
        B[2] = 'B'
        B[3] = j
        B[0] += B[1].count(0)
        candidate.append(A)
        candidate.append(B)
    candidate.sort(key = lambda s: -s[0])
    Beam[i+1] = copy.deepcopy(candidate[:5])

current = 0
Answer = [None] * T
for i in reversed(range(T)):
    Answer[i] = Beam[i+1][current][2]
    current = Beam[i+1][current][3]
for c in Answer:
    print(c)

A
B
A


## 7.5 チャレンジ問題

In [ ]:
# A50
import numpy as np
import random
import time
import sys

N = 100
Q = 1000
A = np.array([ list(map(int, input().split())) for i in range(N) ])

X = [ random.randint(0, N - 1) for i in range(Q) ]
Y = [ random.randint(0, N - 1) for i in range(Q) ]
H = [ 1 ] * Q
B = np.zeros((3 * N, 3 * N))
for i in range(Q):
	B[Y[i]][X[i]] += 1

delta = [ None ] * (N + 1)
for i in range(1, N + 1):
	delta[i] = np.array([ [ max(i - abs(y) - abs(x), 0) for x in range(-i + 1, i) ] for y in range(-i + 1, i) ])

def get_score():
	return 200000000 - np.absolute(A - B[N:2*N, N:2*N]).sum()

TIME_LIMIT = 5.4
current_score = get_score()
ti = time.time()

loops = 0
while time.time() - ti < TIME_LIMIT:
	t = random.randint(0, Q - 1)
	old_x, new_x = X[t], X[t] + random.randint(-9, +9)
	old_y, new_y = Y[t], Y[t] + random.randint(-9, +9)
	old_h, new_h = H[t], H[t] + random.randint(-19, +19)
	if new_x < 0 or new_x >= N or new_y < 0 or new_y >= N or new_h <= 0 or new_h > N:
		continue
	B[N+Y[t]-H[t]+1:N+Y[t]+H[t], N+X[t]-H[t]+1:N+X[t]+H[t]] -= delta[H[t]]
	X[t], Y[t], H[t] = new_x, new_y, new_h
	B[N+Y[t]-H[t]+1:N+Y[t]+H[t], N+X[t]-H[t]+1:N+X[t]+H[t]] += delta[H[t]]
	new_score = get_score()
	if current_score < new_score:
		current_score = new_score
	else:
		B[N+Y[t]-H[t]+1:N+Y[t]+H[t], N+X[t]-H[t]+1:N+X[t]+H[t]] -= delta[H[t]]
		X[t], Y[t], H[t] = old_x, old_y, old_h
		B[N+Y[t]-H[t]+1:N+Y[t]+H[t], N+X[t]-H[t]+1:N+X[t]+H[t]] += delta[H[t]]
	loops += 1

print(Q)
for i in range(Q):
	print(X[i], Y[i], H[i])
print("score =", current_score, file = sys.stderr)
print("loops =", loops, file = sys.stderr)